In [ ]:
#This is a demo code for NLP LDA modeling

# Import

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
import re
import warnings
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation as LDA

Import File

In [ ]:
import pandas as pd
import numpy as np

#data = pd.read_csv('abcnews-date-text.csv')
data=pd.read_excel('topic_modeling.xlsx')

In [ ]:
data.head()

,publish_date,headline_text
0,20030219,aba decides against community broadcasting lic...
1,20030219,act fire witnesses must be aware of defamation
2,20030219,a g calls for infrastructure protection summit
3,20030219,air nz staff in aust strike for pay rise
4,20030219,air nz strike to affect australian travellers


# Data Cleaning

In [ ]:
import nltk
nltk.download('stopwords')
WPT = nltk.WordPunctTokenizer()
stop_word_list = nltk.corpus.stopwords.words('english')
print(stop_word_list)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'ea

In [ ]:
data=data.drop('publish_date', axis=1)
data=data['headline_text']

In [ ]:
data = data.map(lambda x: re.sub('[,\.!?();:$%&#"]', '', x))
data = data.map(lambda x: x.lower())
data = data.map(lambda x: x.strip())

In [ ]:
def token(values):
    filtered_words = [word for word in values.split() if word not in stop_word_list]
    not_stopword_doc = " ".join(filtered_words)
    return not_stopword_doc

data = data.map(lambda x: token(x))

In [ ]:
dataDoc = data.values.tolist()

# Vectorizer (To obtain appropriate variable for LDA)

In [ ]:
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=10000)
tf = tf_vectorizer.fit_transform(dataDoc)

In [ ]:
tf_feature_names = tf_vectorizer.get_feature_names()

# LDA Modeling

In [ ]:
lda = LDA(n_components=30, max_iter=5, learning_method='online', learning_offset=50.,random_state=0).fit(tf)

Showing topics

In [124]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic %d:" % (topic_idx))
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[-10:]]))
display_topics(lda,tf_feature_names,30)

Topic 0:
baby men hits michael young street children brisbane back fire
Topic 1:
sea art group life west man murder north nt charged
Topic 2:
mother govt wants party show pm people child election nsw
Topic 3:
central despite parliament laws concerns arrested take act hospital qld
Topic 4:
ice footage aussie dollar airport four top test perth interview
Topic 5:
students media wins abc house 19 could canberra south queensland
Topic 6:
return australias future review climate league funding big call death
Topic 7:
charges tax commission face time royal guilty final attack police
Topic 8:
opposition debate security plans shooting ban say business sex two
Topic 9:
vote river farm premier women indigenous council rural victoria sa
Topic 10:
gas victims housing friday inquiry public power bushfire set government
Topic 11:
share make east search road tasmanian market gold year coast
Topic 12:
takes rise chinese high minister calls covid australian new coronavirus
Topic 13:
gets david jail park 

# Visualization with pyLDAvis

In [33]:
!pip install pyLDAvis==3.2.2 #other versions does not work

import pyLDAvis.sklearn

pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(lda,tf, tf_vectorizer, mds = 'tsne' )
pyLDAvis.display(panel)

     |████████████████████████████████| 1.7MB 6.9MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-3.2.2-py2.py3-none-any.whl size=135593 sha256=2d3ca98f92bf1db6b5648e851adf9835fb34f1dbe74cf52f4d8a2890a154b5bb
  Stored in directory: /root/.cache/pip/wheels/74/df/b6/97234c8446a43be05c9a8687ee0db1f1b5ade5f27729187eae
Successfully built pyLDAvis


/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/usr/local/lib/python3.7/dist-packages/past/builtins/misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping


# Testing

In [125]:
unseen_test=["For next generations, we may have no future due to the climate changes"]

In [126]:
unseen_test=pd.Series(unseen_test)
unseen_test = unseen_test.map(lambda x: re.sub('[,\.!?();:$%&#"]', '', x))
unseen_test = unseen_test.map(lambda x: x.lower())
unseen_test = unseen_test.map(lambda x: x.strip())
unseen_test = unseen_test.map(lambda x: token(x))
unseen_test = unseen_test.values.tolist()
tf_test=tf_vectorizer.transform(unseen_test)
result_test=lda.transform(tf_test)
result_id=result_test.argmax()

<input>:2: DeprecationWarning: invalid escape sequence \.
<input>:2: DeprecationWarning: invalid escape sequence \.
<input>:2: DeprecationWarning: invalid escape sequence \.
<input>:2: DeprecationWarning: invalid escape sequence \.
<input>:2: DeprecationWarning: invalid escape sequence \.
<input>:2: DeprecationWarning: invalid escape sequence \.
<input>:2: DeprecationWarning: invalid escape sequence \.
<input>:2: DeprecationWarning: invalid escape sequence \.
<ipython-input-126-1f54306c7c42>:2: DeprecationWarning: invalid escape sequence \.
  unseen_test = unseen_test.map(lambda x: re.sub('[,\.!?();:$%&#"]', '', x))


In [127]:
unseen_test

['next generations may future due climate changes']

In [128]:
result_id

6

In [129]:
print([tf_vectorizer.get_feature_names()[j] for j in lda.components_[result_id].argsort()[-10:]])

['return', 'australias', 'future', 'review', 'climate', 'league', 'funding', 'big', 'call', 'death']
